# Modelo SIR


- Suceptible ($\frac{dS}{dt} = - \frac{\beta  S  I}{N}$)
- Infectado  ($\frac{dI}{dt} = \frac{\beta  S  I}{N} - \gamma  I$)
- Recuperado ($\frac{dR}{dt} = - \gamma  I$) 

In [3]:
import ipywidgets as widgets
from ipywidgets import interact 
import numpy as np 
from scipy.integrate import odeint
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def SuceptiblesIniciales(poblacion_total,VI_infectados,VI_recuperados):
    return poblacion_total - (VI_infectados + VI_recuperados)

def CalcularBeta(promedio_de_contactos,probabilidad_de_transmicion):
    return promedio_de_contactos*probabilidad_de_transmicion

def CalcularGamma(dias_de_recuperacion):
    return 1.0/dias_de_recuperacion

def deriv_suceptible(beta,suceptible,infectado,poblacion_total):
    return -beta * suceptible *infectado / poblacion_total

def deriv_infectado(beta,suceptible,infectado,poblacion_total,gamma):
    return beta * suceptible * infectado / poblacion_total - gamma*infectado

def deriv_recuperados(infectado,gamma):
    return gamma*infectado

def deriv_helper(VI,tiempo,poblacion_total,beta,gamma):
    suceptible,infectado,recuperado = VI

    dSdt = deriv_suceptible(beta, suceptible, infectado, poblacion_total)
    dIdt = deriv_infectado(beta, suceptible, infectado, poblacion_total, gamma)
    dRdt = deriv_recuperados(infectado, gamma)

    return dSdt, dIdt, dRdt
@interact(poblacion_total = (0,1000000),dias = (0,100),VI_infectados = (0,100),VI_recuperados = (0,100),promedio_de_contactos = (0,30),probabilidad_de_transmicion = (0.0,1.00),dias_de_recuperacion = (1,30))
def plot_sir(poblacion_total,dias,VI_infectados,VI_recuperados,promedio_de_contactos,probabilidad_de_transmicion,dias_de_recuperacion):
    #suceptibles iniciales
    VI_suceptibles = SuceptiblesIniciales(poblacion_total, VI_infectados, VI_recuperados)
    #beta 
    beta = CalcularBeta(promedio_de_contactos, probabilidad_de_transmicion)
    #gamma
    gamma = CalcularGamma(dias_de_recuperacion)
    #tiempo
    tiempo = np.linspace(0,dias,dias)
    #condiciones iniciales
    VI = (VI_suceptibles,VI_infectados,VI_recuperados)
    #integramos
    func_integ = odeint(deriv_helper, VI, tiempo, args = (poblacion_total,beta,gamma))
    suceptible,infectado,recuperado = func_integ.T
    
    fig = plt.figure(figsize = (10,6.5),facecolor = 'w')
    ax = fig.add_subplot(111,axisbelow=True)
    ax.plot(tiempo, suceptible, 'b', alpha=0.5,lw=2, label = "Suceptible")
    ax.plot(tiempo, infectado , 'r', alpha=0.5,lw=2, label = "Infectado ")
    ax.plot(tiempo, recuperado, 'g', alpha=0.5,lw=2, label = "Recuperado")

    ax.set_xlabel("Días")
    ax.set_ylabel("Poblacion")
    ax.set_title("Modelo SIR")
    legend = ax.legend()

    plt.grid()
    plt.show()

interactive(children=(IntSlider(value=500000, description='poblacion_total', max=1000000), IntSlider(value=182…